<a href="https://colab.research.google.com/github/JSJeong-me/LGE-PRI-1st/blob/main/SuperGlue/SuperGlue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SuperGlue for Visual Place Recognition

In [ ]:
# 1) Colab 런타임을 Python 3, GPU(TPU 아님)로 설정하세요.
#    (상단 메뉴 → 런타임 → 런타임 유형 변경 → 하드웨어 가속기: GPU)

# 2) GitHub 저장소를 클론합니다.
!git clone https://github.com/jomariya23156/SuperGlue-for-Visual-Place-Recognition.git
%cd SuperGlue-for-Visual-Place-Recognition

In [ ]:
# 기본 요구 패키지 설치
!pip install --upgrade pip
!pip install numpy pandas opencv-python torch torchvision matplotlib fastapi uvicorn

# 만약 CUDA 버전의 PyTorch가 필요하면,
# Colab 런타임이 제공하는 CUDA 버전에 맞춰 설치합니다. 예를 들어:
# !pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu117


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 예: 내 Drive 아래에 데이터 폴더를 생성
!mkdir -p /content/drive/MyDrive/VPR_data/frames
!mkdir -p /content/drive/MyDrive/VPR_data/superpoints
!mkdir -p /content/drive/MyDrive/VPR_data/api_output


In [ ]:
# 예시: Drive 내에 저장된 비디오 파일을 프레임으로 추출
!python image_from_video.py \
  --input /content/drive/MyDrive/VPR_data/video.mp4 \
  --output /content/drive/MyDrive/VPR_data/frames \
  --skip 60 \
  --format png


SuperPoint 키포인트/디스크립터 추출

In [ ]:
!python superpoints_from_images.py \
  --input_dir /content/drive/MyDrive/VPR_data/frames \
  --output_dir /content/drive/MyDrive/VPR_data/superpoints \
  --resize 320 240 \
  --max_keypoints 1024 \
  --keypoint_threshold 0.005 \
  --nms_radius 4 \
  --cuda


A. SuperPoint .pickle 파일 기반

In [ ]:
!python superglue_rank_superpoints_file.py \
  --query 0001.pickle \
  --input_dir /content/drive/MyDrive/VPR_data/superpoints \
  --output_dir /content/drive/MyDrive/VPR_data/api_output \
  --image_size 320 240 \
  --superglue indoor \
  --sinkhorn_iterations 20 \
  --match_threshold 0.2 \
  --cuda


B. 이미지 파일 기반 (대안)

In [ ]:
!python superglue_rank_images.py \
  --query 0001.png \
  --input_dir /content/drive/MyDrive/VPR_data/frames \
  --output_dir /content/drive/MyDrive/VPR_data/api_output \
  --resize 320 240 \
  --resize_float \
  --max_keypoints 1024 \
  --keypoint_threshold 0.005 \
  --nms_radius 4 \
  --superglue indoor \
  --sinkhorn_iterations 20 \
  --match_threshold 0.2 \
  --viz \
  --fast_viz \
  --show_keypoints \
  --cuda


결과 시각화

In [ ]:
!python ranking_viz.py \
  --query 0001.png \
  --input_csv /content/drive/MyDrive/VPR_data/api_output/ranking_score.csv \
  --input_dir /content/drive/MyDrive/VPR_data/frames \
  --input_extension png \
  --output_extension png \
  --rank 5

In [ ]:
# 1) ngrok 설치
!pip install pyngrok

# 2) ngrok 토큰 설정 (https://dashboard.ngrok.com/get-started/your-authtoken)
from pyngrok import ngrok
ngrok.set_auth_token("YOUR_NGROK_AUTHTOKEN")

# 3) FastAPI 서버 실행 및 외부 포트 노출
get_ipython().system_raw("uvicorn place_recognition_api:app --host 0.0.0.0 --port 8000 &")
public_url = ngrok.connect(8000)
print("Public URL:", public_url)